In [1]:
%pylab inline
from classy import *

Populating the interactive namespace from numpy and matplotlib
Version:  0.0.16


In [2]:
data=load_excel('data/iris.xls')
data_train,data_test=split(data,test_size=0.2)

iris.data 151 5
150 vectors of length 4
Feature names: 'petal length in cm', 'petal width in cm', 'sepal length in cm', 'sepal width in cm'
Target values given.
Target names: 'Iris-setosa', 'Iris-versicolor', 'Iris-virginica'
Mean:  [ 3.75866667  1.19866667  5.84333333  3.054     ]
Median:  [ 4.35  1.3   5.8   3.  ]
Stddev:  [ 1.75852918  0.76061262  0.82530129  0.43214658]
Original vector shape:  (150, 4)
Train vector shape:  (120, 4)
Test vector shape:  (30, 4)


In [3]:
X=data.vectors
Y=data.targets

In [4]:
from keras.utils import np_utils

Using TensorFlow backend.


In [37]:
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(data_train.targets)

In [38]:
dummy_y[:10,:]

array([[ 0.,  1.,  0.],
       [ 0.,  0.,  1.],
       [ 0.,  0.,  1.],
       [ 0.,  1.,  0.],
       [ 0.,  0.,  1.],
       [ 1.,  0.,  0.],
       [ 0.,  1.,  0.],
       [ 0.,  1.,  0.],
       [ 0.,  0.,  1.],
       [ 0.,  1.,  0.]])

In [39]:
dummy_y.shape

(120, 3)

In [40]:
from keras.models import Sequential
from keras.layers import Dense
import numpy
# fix random seed for reproducibility
numpy.random.seed(7)

In [41]:
data_train.vectors.shape

(120, 4)

In [42]:
data_train.vectors.shape

(120, 4)

In [43]:
# create model
model = Sequential()
model.add(Dense(12, input_dim=data_train.vectors.shape[1], activation='relu'))
model.add(Dense(4, activation='relu'))
model.add(Dense(dummy_y.shape[1], activation='sigmoid'))

In [44]:
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [49]:
model.fit(data_train.vectors, dummy_y,epochs=300,batch_size=10,verbose=False)

In [50]:
# evaluate the model
scores = model.evaluate(data_train.vectors, dummy_y)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

 32/120 [=======>......................] - ETA: 0s
acc: 100.00%


In [56]:
model.predict(data_test.vectors)

array([[  1.00000000e+00,   1.60468385e-08,   4.28483546e-22],
       [  6.10316520e-11,   1.71292470e-06,   9.99998569e-01],
       [  1.00000000e+00,   1.54111817e-08,   5.62919580e-22],
       [  1.00000000e+00,   4.69757255e-09,   9.24069219e-24],
       [  9.99999881e-01,   2.55046928e-07,   3.28012737e-21],
       [  1.33636902e-09,   1.26263112e-01,   8.76937509e-01],
       [  8.55412755e-11,   8.20309469e-06,   9.99994636e-01],
       [  1.00000000e+00,   1.60468385e-08,   4.28483546e-22],
       [  5.34081113e-10,   7.07869003e-06,   9.99994040e-01],
       [  1.00000000e+00,   5.95114291e-10,   7.02447898e-24],
       [  1.53222857e-08,   9.99998093e-01,   1.81717212e-07],
       [  1.00000000e+00,   3.83528848e-10,   7.92520382e-25],
       [  1.00000000e+00,   2.31852249e-09,   1.46577077e-23],
       [  1.00000000e+00,   2.51591525e-10,   1.79693960e-25],
       [  6.13479016e-12,   5.33209764e-04,   9.99574840e-01],
       [  3.37093624e-08,   9.99574006e-01,   3.2729396

In [59]:
out=model.predict(data_test.vectors)
argmax(out,axis=1)

30

In [104]:
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils

class KerasMLP(object):
    
    def percent_correct(self,vectors,targets):
        return self.score(vectors,targets)*100.0
    def __init__(self,**kwargs):
        
        self.model=None
        self.dummy_y=None
        self.hidden_layer_sizes=kwargs.get('hidden_layer_sizes',[4])
    
    
        model.add(Dense(12, input_dim=data_train.vectors.shape[1], activation='relu'))    
                
    def fit(self,*args,**kwargs):
        X,Y=args[0],args[1]
        epochs=kwargs.get('epochs',300)
        batch_size=kwargs.get('batch_size',10)
        if self.model is None:
            self.model=Sequential()
            self.model.add(Dense(self.hidden_layer_sizes[0], input_dim=X.shape[1], activation='relu'))

            for n in self.hidden_layer_sizes[1:]:
                self.model.add(Dense(n, activation='relu'))     
                
            self.dummy_y = np_utils.to_categorical(Y)                
            self.model.add(Dense(self.dummy_y.shape[1], activation='sigmoid'))
            self.model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
            
        self.model.fit(X, dummy_y,epochs=epochs,batch_size=batch_size,verbose=False)
     
    
        self.weights=[]
        self.bias=[]
        
        for layer in self.model.layers:
            w,b = layer.get_weights()     
            self.weights.append(w)
            self.bias.append(b)
    
    
    def percent_correct(self,vectors,targets):
        dummy_y = np_utils.to_categorical(targets)
        scores = self.model.evaluate(vectors, dummy_y,verbose=False)
        return scores[1]

    def output(self, X):
        return self.model.predict(X)

    def predict(self, X):
        output=self.model.predict(X)
        return argmax(output,axis=1)
    
    def predict_names(self,vectors,names):
        result=self.predict(vectors)
        return [names[i] for i in result]
    

In [96]:
C=KerasMLP(hidden_unit_sizes=[4])

In [97]:
timeit(reset=True)
C.fit(data_train.vectors,data_train.targets)
print(("Training time: ",timeit()))

Time Reset
('Training time: ', '6.831284046173096 seconds ')


In [98]:
print(("On Training Set:",C.percent_correct(data_train.vectors,data_train.targets)))
print(("On Test Set:",C.percent_correct(data_test.vectors,data_test.targets)))

('On Training Set:', 0.99444444974263513)
('On Test Set:', 0.95555561780929565)


In [99]:
C.predict(data_train.vectors)

array([1, 2, 2, 1, 2, 0, 1, 1, 2, 1, 0, 0, 2, 2, 0, 1, 2, 2, 2, 1, 0, 2, 0,
       0, 1, 2, 0, 0, 1, 2, 1, 0, 1, 2, 0, 2, 2, 2, 0, 2, 2, 1, 1, 2, 1, 1,
       2, 1, 2, 1, 0, 1, 0, 0, 0, 0, 2, 1, 0, 0, 2, 1, 2, 0, 1, 1, 1, 2, 1,
       1, 1, 0, 0, 0, 0, 2, 1, 1, 2, 2, 0, 2, 1, 0, 2, 0, 0, 0, 2, 1, 2, 1,
       1, 2, 2, 2, 0, 2, 0, 1, 1, 2, 1, 1, 0, 2, 0, 1, 1, 1, 1, 2, 2, 1, 0,
       1, 0, 2, 0, 2])

In [100]:
data_train.targets

array([1, 2, 2, 1, 2, 0, 1, 1, 2, 1, 0, 0, 2, 2, 0, 1, 2, 2, 2, 1, 0, 2, 0,
       0, 1, 2, 0, 0, 1, 1, 1, 0, 1, 2, 0, 2, 2, 2, 0, 2, 2, 1, 1, 2, 1, 1,
       2, 1, 2, 1, 0, 1, 0, 0, 0, 0, 2, 1, 0, 0, 2, 1, 2, 0, 1, 1, 1, 2, 1,
       1, 1, 0, 0, 0, 0, 2, 1, 1, 2, 2, 0, 2, 1, 0, 2, 0, 0, 0, 2, 1, 2, 1,
       1, 2, 2, 2, 0, 2, 0, 1, 1, 2, 1, 1, 0, 2, 0, 1, 1, 1, 1, 2, 2, 1, 0,
       1, 0, 2, 0, 2])

In [103]:
C.weights,C.bias

([array([[-0.93730491, -0.63207489, -1.20870042,  0.83237678],
         [-0.49014619, -0.60080981, -1.70733559,  1.71018529],
         [ 0.63286042, -0.46314052,  0.7151987 , -0.05413043],
         [ 0.5427053 , -0.01360828,  0.7378639 ,  0.21850841]], dtype=float32),
  array([[ 0.74035454,  2.06471992, -3.47726083],
         [-0.1860289 , -0.73370296, -0.73682344],
         [ 1.8240757 , -3.11415005, -0.85160005],
         [-1.31214011, -0.40642127,  0.50792986]], dtype=float32)],
 [array([ 0.89955032,  0.        ,  0.42720661, -0.2098252 ], dtype=float32),
  array([-0.44809642,  0.64235866,  0.08107103], dtype=float32)])